# the q table only calculates 19 different candidates!!!

In [8]:
import numpy as np
import copy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import random
import time
import math
import os
from rouge import Rouge
import pandas as pd

Constants

In [9]:
data = pd.read_csv('./TestData/test.csv')
#limitation = 2
alpha = 0.5
gamma = 0.5
ls = 0.9
epsilon = 0.7

Functions

In [10]:
def calculate_score(candidateSummary,similarityVect,ls):#ls is the trade-off parameter between relevance and redundancy
    score = 0
    reletivity = 0
    redundancy = 0
    #similarityVect = cosine_similarity(docMat)
    for i in candidateSummary:
        reletivity += sum(similarityVect[i]) + (1/(i+1))
        #print(sum(similarityVect[i]))
        for j in candidateSummary:
            if candidateSummary.index(j) > candidateSummary.index(i):
                redundancy += similarityVect[i,j]

    score = (ls * reletivity) - ( (1-ls) * redundancy )
    return score

In [11]:
def find_action(state):
    if len(q_table[state]) == 0:
        next_state = list(state)        
        k = random.randint(0,docMat.shape[0]-1)
        while next_state[k] == '1':
            k = random.randint(0,docMat.shape[0]-1)        
        next_state[k] = 1
        next_state = "".join(str(s) for s in next_state)
    else:        
        if random.random() < epsilon:
            next_state = list(state)            
            k = random.randint(0,docMat.shape[0]-1)
            while next_state[k] == '1':
                k = random.randint(0,docMat.shape[0]-1)            
            next_state[k] = 1
            next_state = "".join(str(s) for s in next_state)
        else:
            max_q = -1
            for i in q_table[state]:
                if q_table[state][i] > max_q:
                    max_q = q_table[state][i]
                    best_state = i
            next_state = best_state
    if next_state not in q_table[state].keys():
        q_table[state][next_state] = 0
    if final_check(next_state) == False:
        if next_state not in q_table.keys():
            q_table[next_state] = {}
    return next_state

In [12]:
def final_check(state):
    if state.count('1') == limitation:
        return True
    else:
        return False

In [13]:
def exploit(state):
    max_q = -1
    for i in q_table[state]:
        if q_table[state][i] > max_q:
            max_q = q_table[state][i]
            best_state = i
            next_state = best_state
    return next_state

In [14]:
scores = {}
summaries = {}
start = time.time()
for data_index in data.index:
    print("working for "+ str(data_index))
    doc = data.loc[data_index].article
    originalSentences = []
    processedSentences = []
    stopWords = set(stopwords.words('english'))
    wnl = WordNetLemmatizer()
    for sentence in doc.split('.'):
        if sentence != '':
            originalSentences.append(sentence)
            processedSentence = ''
            tokens = nltk.word_tokenize(sentence)
            for i in tokens:
                if i not in stopWords and i.isalpha():
                    processedSentence += ' '+wnl.lemmatize(i)
            processedSentences.append(processedSentence)
    cv = CountVectorizer()
    docVec = cv.fit_transform(processedSentences)
    docMat = docVec.toarray()
    
    limitation = int(len(originalSentences) * 0.4)
    
    q_table = {}
    initial_state = "".join(str(s) for s in (np.zeros(docMat.shape[0],dtype=int)))
    q_table[initial_state] = {}            
    similarityVect = cosine_similarity(docMat)
    if limitation >= 1:
        cap = int(math.pow(2,min(docMat.shape[0],15)))
        for r in range(0,cap):
            prev_state = initial_state
            repeat_flag = True
            while repeat_flag:
                reward = 0
                next_state = find_action(prev_state)
                if final_check(next_state):
                    candidate = []
                    for i,v in enumerate(next_state):
                        if v == '1':
                            candidate.append(i)
                    reward = calculate_score(candidate,similarityVect,ls)
                    repeat_flag = False
                    next_best_q = 0
                else:
                    if len(q_table[next_state]) != 0:
                        next_best_q = max(q_table[next_state].values())                
                    else:
                        next_best_q = 0 
                delta = (reward + (gamma * next_best_q) - q_table[prev_state][next_state])
                q_table[prev_state][next_state] += alpha * delta      
                prev_state = next_state                

        prev_state = initial_state
        flag = True
        while flag:
            next_state = exploit(prev_state)
            if final_check(next_state):
                flag = False
            prev_state = next_state

        x = ''
        for i,v in enumerate(prev_state):    
            if v == '1':        
                #print(originalSentences[i])
                x += (originalSentences[i] + '.')

        rouge = Rouge()    
        doc = data.loc[data_index].highlights
        score = rouge.get_scores(x, doc)
        scores[data_index] = score  
        summaries[data_index] = x
    else:
        scores[data_index] = 0  
        summaries[data_index] = ''
    
print(time.time() - start)

working for 0
working for 1
working for 2
working for 3
working for 4
working for 5
working for 6
working for 7
working for 8
working for 9
working for 10
working for 11
working for 12
working for 13
working for 14
working for 15
working for 16
working for 17
working for 18
working for 19
working for 20
working for 21
working for 22
working for 23
working for 24
working for 25
working for 26
working for 27
working for 28
working for 29
working for 30
working for 31
working for 32
working for 33
working for 34
working for 35
working for 36
working for 37
working for 38
working for 39
working for 40
working for 41
working for 42
working for 43
working for 44
working for 45
working for 46
working for 47
working for 48
working for 49
working for 50
working for 51
working for 52
working for 53
working for 54
working for 55
working for 56
working for 57
working for 58
working for 59
working for 60
working for 61
working for 62
working for 63
working for 64
working for 65
working for 66
worki

working for 519
working for 520
working for 521
working for 522
working for 523
working for 524
working for 525
working for 526
working for 527
working for 528
working for 529
working for 530
working for 531
working for 532
working for 533
working for 534
working for 535
working for 536
working for 537
working for 538
working for 539
working for 540
working for 541
working for 542
working for 543
working for 544
working for 545
working for 546
working for 547
working for 548
working for 549
working for 550
working for 551
working for 552
working for 553
working for 554
working for 555
working for 556
working for 557
working for 558
working for 559
working for 560
working for 561
working for 562
working for 563
working for 564
working for 565
working for 566
working for 567
working for 568
working for 569
working for 570
working for 571
working for 572
working for 573
working for 574
working for 575
working for 576
working for 577
working for 578
working for 579
working for 580
working 

working for 1030
working for 1031
working for 1032
working for 1033
working for 1034
working for 1035
working for 1036
working for 1037
working for 1038
working for 1039
working for 1040
working for 1041
working for 1042
working for 1043
working for 1044
working for 1045
working for 1046
working for 1047
working for 1048
working for 1049
working for 1050
working for 1051
working for 1052
working for 1053
working for 1054
working for 1055
working for 1056
working for 1057
working for 1058
working for 1059
working for 1060
working for 1061
working for 1062
working for 1063
working for 1064
working for 1065
working for 1066
working for 1067
working for 1068
working for 1069
working for 1070
working for 1071
working for 1072
working for 1073
working for 1074
working for 1075
working for 1076
working for 1077
working for 1078
working for 1079
working for 1080
working for 1081
working for 1082
working for 1083
working for 1084
working for 1085
working for 1086
working for 1087
working for 10

working for 1512
working for 1513
working for 1514
working for 1515
working for 1516
working for 1517
working for 1518
working for 1519
working for 1520
working for 1521
working for 1522
working for 1523
working for 1524
working for 1525
working for 1526
working for 1527
working for 1528
working for 1529
working for 1530
working for 1531
working for 1532
working for 1533
working for 1534
working for 1535
working for 1536
working for 1537
working for 1538
working for 1539
working for 1540
working for 1541
working for 1542
working for 1543
working for 1544
working for 1545
working for 1546
working for 1547
working for 1548
working for 1549
working for 1550
working for 1551
working for 1552
working for 1553
working for 1554
working for 1555
working for 1556
working for 1557
working for 1558
working for 1559
working for 1560
working for 1561
working for 1562
working for 1563
working for 1564
working for 1565
working for 1566
working for 1567
working for 1568
working for 1569
working for 15

working for 1994
working for 1995
working for 1996
working for 1997
working for 1998
working for 1999
working for 2000
working for 2001
working for 2002
working for 2003
working for 2004
working for 2005
working for 2006
working for 2007
working for 2008
working for 2009
working for 2010
working for 2011
working for 2012
working for 2013
working for 2014
working for 2015
working for 2016
working for 2017
working for 2018
working for 2019
working for 2020
working for 2021
working for 2022
working for 2023
working for 2024
working for 2025
working for 2026
working for 2027
working for 2028
working for 2029
working for 2030
working for 2031
working for 2032
working for 2033
working for 2034
working for 2035
working for 2036
working for 2037
working for 2038
working for 2039
working for 2040
working for 2041
working for 2042
working for 2043
working for 2044
working for 2045
working for 2046
working for 2047
working for 2048
working for 2049
working for 2050
working for 2051
working for 20

...

working for 9225
working for 9226
working for 9227
working for 9228
working for 9229
working for 9230
working for 9231
working for 9232
working for 9233
working for 9234
working for 9235
working for 9236
working for 9237
working for 9238
working for 9239
working for 9240
working for 9241
working for 9242
working for 9243
working for 9244
working for 9245
working for 9246
working for 9247
working for 9248
working for 9249
working for 9250
working for 9251
working for 9252
working for 9253
working for 9254
working for 9255
working for 9256
working for 9257
working for 9258
working for 9259
working for 9260
working for 9261
working for 9262
working for 9263
working for 9264
working for 9265
working for 9266
working for 9267
working for 9268
working for 9269
working for 9270
working for 9271
working for 9272
working for 9273
working for 9274
working for 9275
working for 9276
working for 9277
working for 9278
working for 9279
working for 9280
working for 9281
working for 9282
working for 92

working for 9707
working for 9708
working for 9709
working for 9710
working for 9711
working for 9712
working for 9713
working for 9714
working for 9715
working for 9716
working for 9717
working for 9718
working for 9719
working for 9720
working for 9721
working for 9722
working for 9723
working for 9724
working for 9725
working for 9726
working for 9727
working for 9728
working for 9729
working for 9730
working for 9731
working for 9732
working for 9733
working for 9734
working for 9735
working for 9736
working for 9737
working for 9738
working for 9739
working for 9740
working for 9741
working for 9742
working for 9743
working for 9744
working for 9745
working for 9746
working for 9747
working for 9748
working for 9749
working for 9750
working for 9751
working for 9752
working for 9753
working for 9754
working for 9755
working for 9756
working for 9757
working for 9758
working for 9759
working for 9760
working for 9761
working for 9762
working for 9763
working for 9764
working for 97

working for 10179
working for 10180
working for 10181
working for 10182
working for 10183
working for 10184
working for 10185
working for 10186
working for 10187
working for 10188
working for 10189
working for 10190
working for 10191
working for 10192
working for 10193
working for 10194
working for 10195
working for 10196
working for 10197
working for 10198
working for 10199
working for 10200
working for 10201
working for 10202
working for 10203
working for 10204
working for 10205
working for 10206
working for 10207
working for 10208
working for 10209
working for 10210
working for 10211
working for 10212
working for 10213
working for 10214
working for 10215
working for 10216
working for 10217
working for 10218
working for 10219
working for 10220
working for 10221
working for 10222
working for 10223
working for 10224
working for 10225
working for 10226
working for 10227
working for 10228
working for 10229
working for 10230
working for 10231
working for 10232
working for 10233
working fo

working for 10635
working for 10636
working for 10637
working for 10638
working for 10639
working for 10640
working for 10641
working for 10642
working for 10643
working for 10644
working for 10645
working for 10646
working for 10647
working for 10648
working for 10649
working for 10650
working for 10651
working for 10652
working for 10653
working for 10654
working for 10655
working for 10656
working for 10657
working for 10658
working for 10659
working for 10660
working for 10661
working for 10662
working for 10663
working for 10664
working for 10665
working for 10666
working for 10667
working for 10668
working for 10669
working for 10670
working for 10671
working for 10672
working for 10673
working for 10674
working for 10675
working for 10676
working for 10677
working for 10678
working for 10679
working for 10680
working for 10681
working for 10682
working for 10683
working for 10684
working for 10685
working for 10686
working for 10687
working for 10688
working for 10689
working fo

working for 11091
working for 11092
working for 11093
working for 11094
working for 11095
working for 11096
working for 11097
working for 11098
working for 11099
working for 11100
working for 11101
working for 11102
working for 11103
working for 11104
working for 11105
working for 11106
working for 11107
working for 11108
working for 11109
working for 11110
working for 11111
working for 11112
working for 11113
working for 11114
working for 11115
working for 11116
working for 11117
working for 11118
working for 11119
working for 11120
working for 11121
working for 11122
working for 11123
working for 11124
working for 11125
working for 11126
working for 11127
working for 11128
working for 11129
working for 11130
working for 11131
working for 11132
working for 11133
working for 11134
working for 11135
working for 11136
working for 11137
working for 11138
working for 11139
working for 11140
working for 11141
working for 11142
working for 11143
working for 11144
working for 11145
working fo

In [15]:
data['score'] = scores.values()
data['summary'] = summaries.values()
data

,id,article,highlights,Unnamed: 3,Unnamed: 4,score,summary
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,14,1918.0,"[{'rouge-1': {'r': 0.3333333333333333, 'p': 0....",' But could crowding on planes lead to more se...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,12,402793.0,"[{'rouge-1': {'r': 0.9117647058823529, 'p': 0....",A drunk teenage boy had to be rescued by secur...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,6,NaN,"[{'rouge-1': {'r': 0.42857142857142855, 'p': 0...",Dougie Freedman is on the verge of agreeing a ...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,14,NaN,"[{'rouge-1': {'r': 0.4864864864864865, 'p': 0....",Liverpool target Neto is also wanted by PSG an...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",34,NaN,"[{'rouge-1': {'r': 0.4897959183673469, 'p': 0....","Scroll down for video . Return: Diane Sawyer,..."
...,...,...,...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,35,NaN,"[{'rouge-1': {'r': 0.7428571428571429, 'p': 0....",The object would have been the heat source fo...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,40,NaN,"[{'rouge-1': {'r': 0.6417910447761194, 'p': 0....",A man facing trial for helping his former love...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,88,NaN,"[{'rouge-1': {'r': 0.625, 'p': 0.0655737704918...",A dozen or more metal implements are arranged ...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,9,NaN,"[{'rouge-1': {'r': 0.5161290322580645, 'p': 0....",Brook Lopez dominated twin brother Robin with ...


In [16]:
scores

{0: [{'rouge-1': {'r': 0.3333333333333333,
    'p': 0.12222222222222222,
    'f': 0.17886178469165187},
   'rouge-2': {'r': 0.09090909090909091,
    'p': 0.022388059701492536,
    'f': 0.03592814054143238},
   'rouge-l': {'r': 0.3333333333333333,
    'p': 0.12222222222222222,
    'f': 0.17886178469165187}}],
 1: [{'rouge-1': {'r': 0.9117647058823529,
    'p': 0.31313131313131315,
    'f': 0.4661654097280797},
   'rouge-2': {'r': 0.5142857142857142,
    'p': 0.13636363636363635,
    'f': 0.2155688589623149},
   'rouge-l': {'r': 0.8529411764705882,
    'p': 0.29292929292929293,
    'f': 0.4360902217581548}}],
 2: [{'rouge-1': {'r': 0.42857142857142855,
    'p': 0.4444444444444444,
    'f': 0.4363636313652893},
   'rouge-2': {'r': 0.16129032258064516,
    'p': 0.1724137931034483,
    'f': 0.16666666167222238},
   'rouge-l': {'r': 0.35714285714285715,
    'p': 0.37037037037037035,
    'f': 0.3636363586380166}}],
 3: [{'rouge-1': {'r': 0.4864864864864865,
    'p': 0.23076923076923078,
    '

In [17]:
sum = 0
n = 0
for i in scores.values():
    if i!=0:
        sum += i[0]['rouge-1']['f']
        n +=1

avg = sum / n
avg

0.2653374124442349

In [18]:
data.to_csv('output.csv')

In [19]:
rouge = Rouge()
score = rouge.get_scores(data.summary, data.highlights, avg=True)
score

ValueError: Hypothesis is empty.